In [156]:
## predic price of car

In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [158]:
df = pd.read_csv("cardekho_imputated.csv")
df.drop("Unnamed: 0", axis = 1 ,inplace = True)

In [159]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


### data cleaning

In [160]:
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [161]:
## droping car name and brand since model has the mian info
df.drop(['car_name','brand'], axis = 1 ,inplace= True)

In [162]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [163]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [164]:
num_features = [features for features in df.columns if df[features].dtype != 'O']
print("numerical features ",len(num_features))
cat_features = [features for features in df.columns if df[features].dtype == 'O']
print("categorical features :", len(cat_features))

discrete_features = [features for features in num_features if df[features].nunique() <= 25]
print("discrete fratures : ",len(discrete_features))

contin_features = [features for features  in num_features if features not in discrete_features]
print("continuous features : ", len(contin_features))

numerical features  7
categorical features : 4
discrete fratures :  2
continuous features :  5


In [165]:
### ind and dep features 
X = df.drop('selling_price', axis =1 )
y = df['selling_price']

In [166]:
# X_train.head()

### feature encoding scaling

In [167]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [168]:
######## category encoding the models
import category_encoders as ce
tagret_encoder = ce.TargetEncoder(cols = ['model'], smoothing= 0.5)
X_train['model'] = tagret_encoder.fit_transform(X_train['model'], y_train)

In [169]:
X_train.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
11210,772120.619728,7,70252,Dealer,Diesel,Automatic,11.20,2400,215.0,5
1347,456487.804878,2,10000,Individual,Petrol,Manual,23.84,1199,84.0,5
10363,772120.619827,2,6000,Dealer,Diesel,Automatic,19.00,1950,241.3,5
316,622130.578512,7,63000,Dealer,Petrol,Manual,17.80,1497,117.3,5
10638,278753.012048,10,80292,Dealer,Petrol,Manual,20.36,1197,78.9,5


In [170]:
X_test['model'] = tagret_encoder.transform(X_test['model'], y_test)

In [171]:
######### transformation for rrest of features
num_features = X.select_dtypes(exclude = 'object').columns
cat_features = X.select_dtypes(include = 'object').columns
onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']

In [172]:
cat_features

Index(['model', 'seller_type', 'fuel_type', 'transmission_type'], dtype='object')

In [173]:
from sklearn.preprocessing import OneHotEncoder , StandardScaler
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop = 'first')


In [174]:

from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(
   [ 
 ("standard sclaer" , numeric_transformer , num_features),
("ohe" , oh_transformer, onehot_columns)
   ], remainder= 'passthrough'
    
)## PASS THROUGH IS VERY IMPORTANT OTHERWISE AL THE COLUMN ON WHCH NOOPERATION PEFORMED WOULD BE REMOVED

In [175]:
X_train = transformer.fit_transform(X_train)

In [176]:
X_test = transformer.transform(X_test)

### model training 

In [186]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score , mean_absolute_error, mean_squared_error

In [197]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rms = np.sqrt(mean_squared_error(true, predicted))
    r2_score_cal= r2_score(true, predicted)
    return mae,  rms, r2_score_cal

In [207]:
models = {
    "Linear Regression": LinearRegression(),
    "lasso" : Lasso(),
    "ridge" : Ridge(),
    "knn" : KNeighborsRegressor(),
    "dt" : DecisionTreeRegressor(),
    "rf"  : RandomForestRegressor()
}
for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    # model preidctons
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rms, model_train_r2 = evaluate_model(y_train_pred, y_train)
    model_test_mae, model_test_rms, model_test_r2 = evaluate_model(y_test_pred, y_test)


    print(list(models.keys())[i])

    print("model performace on training set: ")
    print(" - root mean squared error {:.4f}".format(model_train_rms))
    print(" - mean absolute error {:.4f}".format(model_train_mae))
    print(" - r2_score {:.4f}".format(model_train_r2))

    print("----------------------------------------------------")

  
    print("model performace on test set: ")
    print(" - root mean squared error {:.4f}".format(model_test_rms))
    print(" - mean absolute error {:.4f}".format(model_test_mae))
    print(" - r2_score {:.4f}".format(model_test_r2))

Linear Regression
model performace on training set: 
 - root mean squared error 553521.9748
 - mean absolute error 262634.2392
 - r2_score 0.3929
----------------------------------------------------
model performace on test set: 
 - root mean squared error 499957.6365
 - mean absolute error 274023.2593
 - r2_score 0.5287
lasso
model performace on training set: 
 - root mean squared error 553521.9792
 - mean absolute error 262632.2049
 - r2_score 0.3929
----------------------------------------------------
model performace on test set: 
 - root mean squared error 499956.7919
 - mean absolute error 274019.5113
 - r2_score 0.5287
ridge
model performace on training set: 
 - root mean squared error 553522.6761
 - mean absolute error 262593.3032
 - r2_score 0.3927
----------------------------------------------------
model performace on test set: 
 - root mean squared error 499946.9975
 - mean absolute error 273957.0995
 - r2_score 0.5287
knn
model performace on training set: 
 - root mean squ

In [218]:
## best 2 models are knn and dt so we would do the parameter tuning on both
knn_params = { "n_neighbors" : [2,3, 4, 6,10]}
rf_params = {
    "max_depth" : [5,8,15,None,10],
    "max_features" : [5,7,"auto",8],
    "min_samples_split" : [2,8,15,20],
     "n_estimators" : [100,200,500,1000]
}

In [219]:
random_cv_models = [
    ("knn", KNeighborsRegressor(), knn_params),
    ("rf", RandomForestRegressor(), rf_params)
]

In [220]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, param in random_cv_models:
    random = RandomizedSearchCV(estimator = model,
                                param_distributions = param,
                                n_iter = 100 ,
                                cv = 3, verbose = 2, n_jobs = -1  
                                
    )
    random.fit(X_train, y_train)
    model_param[name]  = random.best_params_

for model_name in model_param:
    print(f"--------------best params -----------for : , {model_name}------------- are :" )
    print(model_param[model_name])

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
--------------best params -----------for : , knn------------- are :
{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 8, 'max_depth': 15}
--------------best params -----------for : , rf------------- are :
{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 8, 'max_depth': 15}


In [229]:
models = {
"random forest regressor" : RandomForestRegressor(n_estimators = 100,min_samples_split =2 , max_features  = 8, max_depth = 15),
"knn" : KNeighborsRegressor(n_neighbors= 4)
}

In [224]:
for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    # make predictions 
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    


--------------best params -----------for : , knn------------- are :
{'n_neighbors': 4}
--------------best params -----------for : , rf------------- are :
{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 8, 'max_depth': 15}


In [230]:
list(models.values())

[RandomForestRegressor(max_depth=15, max_features=8),
 KNeighborsRegressor(n_neighbors=4)]

In [232]:
models = {
    "random forest" : RandomForestRegressor(max_depth=15, max_features=8),
    "knn": KNeighborsRegressor(n_neighbors=4)
}
for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    # make predictions 
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    model_train_mae, model_train_rms, model_train_r2 = evaluate_model(y_train_pred, y_train)
    model_test_mae, model_test_rms, model_test_r2 = evaluate_model(y_test_pred, y_test)


    print(list(models.keys())[i])

    print("model performace on training set: ")
    print(" - root mean squared error {:.4f}".format(model_train_rms))
    print(" - mean absolute error {:.4f}".format(model_train_mae))
    print(" - r2_score {:.4f}".format(model_train_r2))

    print("----------------------------------------------------")

  
    print("model performace on test set: ")
    print(" - root mean squared error {:.4f}".format(model_test_rms))
    print(" - mean absolute error {:.4f}".format(model_test_mae))
    print(" - r2_score {:.4f}".format(model_test_r2))

random forest
model performace on training set: 
 - root mean squared error 136850.1308
 - mean absolute error 52333.1093
 - r2_score 0.9736
----------------------------------------------------
model performace on test set: 
 - root mean squared error 206244.4781
 - mean absolute error 95754.7965
 - r2_score 0.9404
knn
model performace on training set: 
 - root mean squared error 304494.6351
 - mean absolute error 83052.6444
 - r2_score 0.8514
----------------------------------------------------
model performace on test set: 
 - root mean squared error 249976.1835
 - mean absolute error 106136.9202
 - r2_score 0.8972
